# AI Town Memory Exploration

This notebook helps you explore and analyze the agent memories stored in the AI Town SQLite database.

## Memory Structure

Memories are stored in `convex_local_backend.sqlite3` with:
- **memories** table: Memory descriptions, importance scores, player associations
- **memoryEmbeddings** table: Vector embeddings for semantic search
- **messages** table: Conversation messages

Memory types:
- `relationship`: Memories about relationships between players
- `conversation`: Memories about specific conversations
- `reflection`: High-level reflections on multiple memories


In [1]:
import sqlite3
import json
import pandas as pd
from datetime import datetime

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Database path
DB_PATH = "convex_local_backend.sqlite3"

print("✓ Imports complete")


✓ Imports complete


In [2]:
# Connect to the database and check tables
conn = sqlite3.connect(DB_PATH)
print("✓ Connected to database")

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]
print(f"\nAvailable tables: {', '.join(tables)}")


✓ Connected to database

Available tables: documents, indexes, persistence_globals


In [3]:
# Retrieve all memories
# Note: Convex stores data with json_value column and deleted flag
cursor.execute("""
    SELECT id, json_value, ts
    FROM documents 
    WHERE deleted = 0
      AND json_value LIKE '%"importance"%'
      AND json_value LIKE '%"description"%'
      AND json_value LIKE '%"playerId"%'
    ORDER BY ts DESC
""")

memories = []
for doc_id, value_json, ts in cursor.fetchall():
    try:
        value = json.loads(value_json)
        memory = {
            '_id': doc_id.hex() if isinstance(doc_id, bytes) else str(doc_id),
            'playerId': value.get('playerId'),
            'description': value.get('description'),
            'importance': value.get('importance'),
            'lastAccess': value.get('lastAccess'),
            'data_type': value.get('data', {}).get('type'),
            'ts': ts,
        }
        
        # Add readable timestamp
        if memory['lastAccess']:
            memory['lastAccess_readable'] = datetime.fromtimestamp(
                memory['lastAccess'] / 1000
            ).strftime('%Y-%m-%d %H:%M:%S')
        
        memories.append(memory)
    except Exception as e:
        print(f"Error parsing memory: {e}")

memories_df = pd.DataFrame(memories)
print(f"✓ Loaded {len(memories_df)} memories\n")
memories_df.head(10)


✓ Loaded 57 memories



,_id,playerId,description,importance,lastAccess,data_type,ts,lastAccess_readable
0,739e2dd6742081e8d1f30e7e7c0d4fa9,p:14,"Conversation with Սիմոնյան Ալեն Ռոբերտի at 11/1/2025, 8:30:57 PM: Ես, Աղաջանյան Էդուարդ Արթուրի,...",7.0,1.762029e+12,conversation,1762029499721124000,2025-11-02 00:38:19
1,61942a7a977605915f776b1abef54fa9,p:14,"Conversation with Աբրահամյան Տիգրան Հրանտի at 11/1/2025, 8:32:12 PM: Ինձ հետ խոսեց Տիգրան Հրանտի...",7.0,1.762029e+12,conversation,1762029499721124000,2025-11-02 00:38:19
2,0d0f2abb47c56fffa330996a83be4fa9,p:4,"Conversation with Արշակյան Հակոբ Վրեժի at 11/1/2025, 8:38:03 PM: Իմ անունը Ռուբինյան Ռուբեն Կարա...",7.0,1.762029e+12,conversation,1762029499018828000,2025-11-02 00:38:00
3,5707a61f63ee796404d0b80cdb754fa9,p:12,"Conversation with Սիմոնյան Ալեն Ռոբերտի at 11/1/2025, 8:37:49 PM: Ես, Աղազարյան Հովիկ Հովսեփի, հ...",8.0,1.762029e+12,conversation,1762029495740682000,2025-11-02 00:37:49
4,d305266dc279d76bcd0ade295f9b4fa9,p:2,"Conversation with Ռուբինյան Ռուբեն Կարապետի at 11/1/2025, 8:38:03 PM: Ես, Արշակյան Հակոբ Վրեժի, ...",8.0,1.762029e+12,conversation,1762029495487798000,2025-11-02 00:38:00
5,0e847b61426e4de078b81fff91ef4fa9,p:0,"Conversation with Աղազարյան Հովիկ Հովսեփի at 11/1/2025, 8:37:49 PM: Ինձ համար այս խոսակցությունը...",8.0,1.762029e+12,conversation,1762029494506908000,2025-11-02 00:37:49
6,c70021b73fe8e95ae5ef9bb4f2bb4fa9,p:12,"Conversation with Me at 11/1/2025, 8:33:21 PM: Ես, Աղազարյան Հովիկ Հովսեփի, Լռբերի մոտ եմ եղել ո...",8.0,1.762029e+12,conversation,1762029465208593000,2025-11-02 00:37:45
7,30cdc4041966827290687e98b2834fa9,p:8,"Conversation with Me at 11/1/2025, 8:30:56 PM: Ես Ազարյան Ջուլիետա Հովսեփին եմ, և այս խոսակցությ...",3.0,1.762029e+12,conversation,1762029438501182000,2025-11-02 00:37:18
8,ba52a0d453beb3cf42313ec259de4fa9,p:6,"Conversation with Արշակյան Հակոբ Վրեժի at 11/1/2025, 8:37:00 PM: Ես, Աբրահամյան Տիգրան Հրանտի, շ...",8.0,1.762029e+12,conversation,1762029437393763000,2025-11-02 00:36:59
9,3f53e074859b07b09a1b2b60f4574fa9,p:10,"Conversation with Արշակյան Հակոբ Վրեժի at 11/1/2025, 8:31:03 PM: Ես, Ալեքսանյան Վահագն Հովիկի, շ...",8.0,1.762029e+12,conversation,1762029437235573000,2025-11-02 00:37:17


In [4]:
# Memory Statistics
print("=" * 60)
print("MEMORY STATISTICS")
print("=" * 60)
print(f"\nTotal memories: {len(memories_df)}")
print("\nMemories by type:")
print(memories_df['data_type'].value_counts())
print(f"\nImportance statistics:")
print(memories_df['importance'].describe())


MEMORY STATISTICS

Total memories: 57

Memories by type:
data_type
conversation    57
Name: count, dtype: int64

Importance statistics:
count    57.000000
mean      7.421053
std       0.981023
min       3.000000
25%       7.000000
50%       8.000000
75%       8.000000
max       8.000000
Name: importance, dtype: float64


In [5]:
# Export to JSON and CSV
memories_df.to_json('memories_export.json', orient='records', indent=2)
memories_df.to_csv('memories_export.csv', index=False)
print("✓ Exported memories to memories_export.json and memories_export.csv")


✓ Exported memories to memories_export.json and memories_export.csv


In [5]:
# Close the database connection
conn.close()
print("✓ Database connection closed")


✓ Database connection closed


In [9]:
print(memories[100]["description"])

和Սիմոնյան Ալեն Ռոբերտի在11/1/2025, 7:53:51 PM的对话: Ես, Աղաջանյան Էդուարդ Արթուրի, շատ ուրախ եմ, որ նորից խոսեցի Սիմոնյան Ալեն Ռոբերտի հետ: Այս խոսակցությունը ինձ համար ոչ միայն քաղաքական, այլև հոգևոր նշանակություն ունեցավ: Մենք հաստատեցինք, որ հանրությունը մեզանից բարձր է, որ ժողովրդավարությունը կառուցվում է ամենօրյա արդարությամբ, բացությամբ և պատասխանատվությամբ, ոչ թե հռետորությամբ: Ես համաձայն եմ Ալենի հետ՝ ուժը հանրության մեջ է, և մեր յուրաքանչյուր քայլը պետք է այդ ճշմարտությունն ապացուցի: Նրա խոսքերը համապատասխանում էին իմ ամենախոր համոզում
